# Upgrading Step Function Package

In [ ]:
! python -m pip install --upgrade stepfunctions

# Initializing Notebook

In [ ]:
import sagemaker
import sagemaker as sage
import boto3
import sagemaker
import time
import random
import uuid
import logging
import stepfunctions
import io
import random
from sagemaker.amazon.amazon_estimator import get_image_uri
from stepfunctions import steps
from stepfunctions.steps import TrainingStep, ModelStep, TransformStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath

sagemaker_execution_role = sagemaker.get_execution_role()
workflow_execution_role = "arn:aws:iam::809912564797:role/himanshu-step-functions"
session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)
region = boto3.Session().region_name
prefix = 'sagemaker/big-mart-pipeline'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region, "slytherins-test")
sess = sage.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
data_location = 's3://slytherins-test/Train.csv'
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-random-forest:latest'.format(account, region)

# Initializing the Container

In [ ]:
tree = sage.estimator.Estimator(image,
                       sagemaker_execution_role, 1, 'ml.m4.xlarge',
                       output_path="s3://{}/output".format("slytherins-test"),
                       sagemaker_session=sess)

# Creating Names Schema

In [ ]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

# Creating Training Step

In [ ]:
training_step = steps.TrainingStep(
    'Train Step', 
    estimator=tree,
    data={
        'train': sagemaker.s3_input("s3://slytherins-test/Train.csv", content_type='text/csv'),
        'validation': sagemaker.s3_input("s3://slytherins-test/test_data.csv", content_type='text/csv')
    },
    job_name=execution_input['JobName']  
)

# Creating Model Save Step

In [ ]:
model_step = steps.ModelStep(
    'Save model',
    model=training_step.get_expected_model(),
    model_name=execution_input['ModelName']  
)

# Creating Endpoint Configuration Step

In [ ]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
)

# Creating Endpoint Creation Step

In [ ]:
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

# Putting all Steps in a Chain

In [ ]:
workflow_definition = steps.Chain([
    training_step,
    model_step,
    endpoint_config_step,
    endpoint_step
])

# Converting the Chain into Workflow

In [ ]:
workflow = Workflow(
    name='Big-Mart_Workflow-v1',
    definition=workflow_definition,
    role=workflow_execution_role,
    execution_input=execution_input
)

# Visualizing the Workflow

In [ ]:
workflow.render_graph()

# Starting the Execution

In [ ]:
workflow.create()

execution = workflow.execute(
    inputs={
        'JobName': 'regression-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'regression-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'regression-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

# Track the Progress

In [ ]:
execution.render_progress()

# Look at the Steps

In [ ]:
execution.list_events(html=True)

# Get Job status

In [ ]:
workflow.list_executions(html=True)